In [1]:
from muselsl import *
import time
import numpy as np
import serial
import matplotlib.pyplot as plt
from pylsl import StreamInlet, resolve_byprop
import warnings
from PsychoTracker import Tracker, SHIFT_LENGTH, NUM_CHANNELS, EPOCH_LENGTH, OVERLAP_LENGTH, BUFFER_LENGTH
import utils  # Our own utility functions
from Process import ChannelDataProcessor
from Metrics import *
from PsychoPy_Code.PsychoRun import run_psychopy

In [2]:
"""
Estimate Relaxation from Band Powers
This example shows how to buffer, epoch, and transform EEG data from a single
electrode into values for each of the classic frequencies (e.g. alpha, beta, theta)
Furthermore, it shows how ratios of the band powers can be used to estimate
mental state for neurofeedback.
The neurofeedback protocols described here are inspired by
*Neurofeedback: A Comprehensive Review on System Design, Methodology and Clinical Applications* by Marzbani et. al
Adapted from https://github.com/NeuroTechX/bci-workshop
"""

# Handy little enum to make code more readable

class Channel:
    TP0 = 0
    FP1 = 1
    FP2 = 2
    TP10 = 3
    DRL = 4


""" 
1. CONNECT TO EEG STREAM 
"""

# Search for active LSL streams

print('Looking for an EEG stream...')
streams = resolve_byprop('type', 'EEG', timeout=2)
if len(streams) == 0:
    raise RuntimeError('Can\'t find EEG stream.')

# Set active EEG stream to inlet and apply time correction

print("Start acquiring data")
inlet = StreamInlet(streams[0], max_chunklen=12)
eeg_time_correction = inlet.time_correction()

# Get the stream info and description
info = inlet.info()
description = info.desc()

"""
2. Obtain Threshold from PsycoPy 
"""

cal = Calibrator(inlet=inlet, info=info, channel_index=Channel.FP1)
cal.start_stage(mode='relax', stage=1)
time.sleep(1)

cal.end_stage(mode='relax', stage=1)

run_psychopy(cmd=cal)
cal.update_stage_threshold()
threshold = 0.0223716588909309062
print("FINAL THRESH", threshold)

"""
3. Record data and wave hand if metric is above threshold.  

"""

# The try/except structure allows to quit the while loop by aborting the
# script with <Ctrl-C>
print('Press Ctrl-C in the console to break the while loop.')

arduinoData = serial.Serial('com3',9600)
time.sleep(1)
try:
    # The following loop acquires data, computes band powers, and calculates neurofeedback metrics based on those band powers
    while True:
        """ 3.1 ACQUIRE DATA """

        # Obtain EEG data from the LSL stream
        eeg_data, timestamp = _acquire_eeg_data(inlet)           
        c = ChannelDataProcessor(buffer_length=BUFFER_LENGTH, epoch_length=EPOCH_LENGTH,
                             overlap_length=OVERLAP_LENGTH, 
                             shift_length=SHIFT_LENGTH, fs=fs, band_cls=Band)

        c.feed_new_data(eeg_data=eeg_data)  # Feed new data generated in the epoch

        metrics = [] # should be 4X8 matrix for each sample
        for i in range(NUM_CHANNELS):  # Iterate through all separate channels

            # Record channel smooth band power
            csbp = c.get_channel_smooth_band_powers(i) 
            # Run calculations on csbp to obtain desired metrics

            if i in [1,2,3]:
                theta_metric = csbp[1]

                metrics.append(theta_metric)
                ratios.append(Metrics.beta_protocol(csbp, Band))
        metric = np.mean(metrics)  
        print(metric)    


        if check_positive(_metric=metric, _threshold=threshold): 

            hand_wave()
        else:
            hand_stay()

except KeyboardInterrupt:
    print('Closing!')   
arduinoData.close()

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html
Looking for an EEG stream...
Start acquiring data
Press Ctrl-C in the console to break the while loop.
average theta: -0.04435660726849113
1
Press Ctrl-C in the console to break the while loop.
average theta: -0.03758421084237624
Press Ctrl-C in the console to break the while loop.
average theta: -0.041048114238721706
Press Ctrl-C in the console to break the while loop.
average theta: -0.039529120865585424
Press Ctrl-C in the console to break the while loop.
average theta: -0.034021134282557874
Press Ctrl-C in the console to break the while loop.
average theta: -0.03814994310260588
Press Ctrl-C in the console to break the while loop.
average theta: -0.039537157012148244
31.8525 	WARNING 	User requested fullscreen with size [1024  768], but screen is actually [1920, 1080]. Using actual size


AttributeError: 'DataRecord' object has no attribute 'avg_thetas'